# Setup

In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None) # display all columns

# Load All Files

### Get filenames from repo

We first retrieve the filenames of all files listed in the repository:

In [3]:
import requests

user = "ard-data"
repo = "2020-rki-archive"

url = "https://api.github.com/repos/{}/{}/git/trees/master?recursive=1".format(user, repo)
r = requests.get(url)
res = r.json()

We only keep the files in the folder data/2_parsed

In [4]:
files_parsed = [file["path"] for file in res["tree"] if file["path"].startswith("data/2_parsed/")]

In [5]:
# the first 5 files in the folder data/2_parsed
files_parsed[:5]

['data/2_parsed/data_2020-03-27-12-00.json.bz2',
 'data/2_parsed/data_2020-03-28-12-00.json.bz2',
 'data/2_parsed/data_2020-03-30-12-00.json.bz2',
 'data/2_parsed/data_2020-03-31-01-37.json.bz2',
 'data/2_parsed/data_2020-04-01-21-19.json.bz2']

In [6]:
files_parsed[-1]

'data/2_parsed/data_2020-06-25-02-31.json.bz2'

### Retrieve latest file from data/2_parsed

In [7]:
df = pd.read_json('https://github.com/ard-data/2020-rki-archive/raw/master/' + files_parsed[-1], compression='bz2')

In [8]:
df.head()

,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,ObjectId,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2,MeldedatumISO,DatenstandISO,RefdatumISO
0,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,14086200,1584144000000,1001,"25.06.2020, 00:00 Uhr",0,-9,1584316800000,0,1,1,Nicht übermittelt,2020-03-14,2020-06-25,2020-03-16
1,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,14086201,1584576000000,1001,"25.06.2020, 00:00 Uhr",0,-9,1584057600000,0,1,1,Nicht übermittelt,2020-03-19,2020-06-25,2020-03-13
2,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,14086202,1584576000000,1001,"25.06.2020, 00:00 Uhr",0,-9,1584316800000,0,1,1,Nicht übermittelt,2020-03-19,2020-06-25,2020-03-16
3,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,14086203,1584748800000,1001,"25.06.2020, 00:00 Uhr",0,-9,1584057600000,0,1,1,Nicht übermittelt,2020-03-21,2020-06-25,2020-03-13
4,1,Schleswig-Holstein,SK Flensburg,A15-A34,M,1,0,14086204,1585267200000,1001,"25.06.2020, 00:00 Uhr",0,-9,1584835200000,0,1,1,Nicht übermittelt,2020-03-27,2020-06-25,2020-03-22


# Cleaning

If the column 'DatenstandISO' is missing, we infer it from 'Datenstand'.

In [9]:
if 'DatenstandISO' not in df.columns:
    print('Column DatenstandISO missing. Infer from column Datenstand.')
    df['DatenstandISO'] = pd.to_datetime(df.Datenstand.str.replace('Uhr', '')).astype(str)
else:
    print('Column DatenstandISO exists.')

Column DatenstandISO exists.


# Aggregation on state level (Bundesländer)

In [10]:
# compute the sum for each date within each state
df_agg = df[df.NeuerTodesfall >= 0].groupby(['DatenstandISO', 'Bundesland'])['AnzahlTodesfall'].sum().reset_index()

In [11]:
df_agg

,DatenstandISO,Bundesland,AnzahlTodesfall
0,2020-06-25,Baden-Württemberg,1827
1,2020-06-25,Bayern,2583
2,2020-06-25,Berlin,212
3,2020-06-25,Brandenburg,165
4,2020-06-25,Bremen,49
5,2020-06-25,Hamburg,259
6,2020-06-25,Hessen,505
7,2020-06-25,Mecklenburg-Vorpommern,20
8,2020-06-25,Niedersachsen,623
9,2020-06-25,Nordrhein-Westfalen,1669


### Add FIPS region codes

We add the corresponding region codes for each state given by https://en.wikipedia.org/wiki/List_of_FIPS_region_codes_(G–I)#GM:_Germany.

In [12]:
state_names = ['Baden-Württemberg', 'Bayern', 'Bremen', 'Hamburg', 'Hessen', 'Niedersachsen', 'Nordrhein-Westfalen', 'Rheinland-Pfalz',
'Saarland', 'Schleswig-Holstein', 'Brandenburg', 'Mecklenburg-Vorpommern', 'Sachsen', 'Sachsen-Anhalt', 'Thüringen', 'Berlin']

gm = ['GM0' + str(i) for i in range(1, 10)] + ['GM' + str(i) for i in range(10, 17)] 

fips_codes = pd.DataFrame({'Bundesland':state_names, 'location':gm})

In [13]:
# add fips codes to dataframe with aggregated data
df_agg = df_agg.merge(fips_codes, left_on='Bundesland', right_on='Bundesland')

### Change location_name to English names

In [14]:
fips_english = pd.read_csv('../../template/base_germany.csv')

In [15]:
df_agg = df_agg.merge(fips_english, left_on='location', right_on='V1')

### Rename columns and sort by date and location

In [16]:
df_agg = df_agg.rename(columns={'DatenstandISO': 'date', 'AnzahlTodesfall': 'value', 'V2':'location_name'})[
    ['date', 'location', 'location_name', 'value']].sort_values(['date', 'location']).reset_index(drop=True)

In [17]:
df_agg

,date,location,location_name,value
0,2020-06-25,GM01,Baden-Württemberg State,1827
1,2020-06-25,GM02,Free State of Bavaria,2583
2,2020-06-25,GM03,Free Hanseatic City of Bremen,49
3,2020-06-25,GM04,Free Hanseatic City of Hamburg,259
4,2020-06-25,GM05,Hesse State,505
5,2020-06-25,GM06,Lower Saxony State,623
6,2020-06-25,GM07,North Rhine-Westphalia State,1669
7,2020-06-25,GM08,Rhineland-Palatinate State,235
8,2020-06-25,GM09,Saarland State,169
9,2020-06-25,GM10,Schleswig-Holstein State,152


In [18]:
df_germany = df_agg.groupby('date')['value'].sum().reset_index()
df_germany['location'] = 'GM'
df_germany['location_name'] = 'Germany'

In [19]:
df_germany.head()

,date,value,location,location_name
0,2020-06-25,8927,GM,Germany


In [20]:
# add data for Germany to dataframe with states
df_cum = pd.concat([df_agg, df_germany]).sort_values(['date', 'location']).reset_index(drop=True)

In [21]:
df_cum.head()

,date,location,location_name,value
0,2020-06-25,GM,Germany,8927
1,2020-06-25,GM01,Baden-Württemberg State,1827
2,2020-06-25,GM02,Free State of Bavaria,2583
3,2020-06-25,GM03,Free Hanseatic City of Bremen,49
4,2020-06-25,GM04,Free Hanseatic City of Hamburg,259


# Load Current Dataframe

In [22]:
df_all = pd.read_csv('../../data-truth/RKI/truth_cum_deaths.csv')

In [23]:
df_all.shape

(1528, 4)

In [24]:
df_all.tail()

,date,location,location_name,value
1523,2020-06-25,GM12,Mecklenburg-Western Pomerania State,20
1524,2020-06-25,GM13,Free State of Saxony,223
1525,2020-06-25,GM14,Sachsen-Anhalt State,58
1526,2020-06-25,GM15,Free State of Thüringia,178
1527,2020-06-25,GM16,Berlin State,212


# Add New Dataframe

In [25]:
df_cum = pd.concat([df_all, df_cum])

In [26]:
df_cum.reset_index(drop=True, inplace=True)

In [27]:
df_cum.shape

(1545, 4)

# Drop Duplicates

In case we accidentally load the same file twice.

In [28]:
df_cum.drop_duplicates(inplace=True)

In [29]:
df_cum.shape

(1528, 4)

# Incidence

We compute the incident deaths as the difference to the previous day.

In [30]:
df_inc = df_cum.copy()

In [31]:
df_inc.value = df_inc.groupby(['location'])['value'].diff()
df_inc.dropna(inplace=True)
df_inc.value = df_inc.value.astype(int)

In [32]:
df_inc.tail(17)

,date,location,location_name,value
1511,2020-06-25,GM,Germany,13
1512,2020-06-25,GM01,Baden-Württemberg State,1
1513,2020-06-25,GM02,Free State of Bavaria,5
1514,2020-06-25,GM03,Free Hanseatic City of Bremen,0
1515,2020-06-25,GM04,Free Hanseatic City of Hamburg,0
1516,2020-06-25,GM05,Hesse State,2
1517,2020-06-25,GM06,Lower Saxony State,1
1518,2020-06-25,GM07,North Rhine-Westphalia State,1
1519,2020-06-25,GM08,Rhineland-Palatinate State,0
1520,2020-06-25,GM09,Saarland State,0


### Export Cum. Deaths

In [33]:
df_cum.to_csv('../../data-truth/RKI/truth_cum_deaths.csv', index=False)

### Export Inc. Deaths

In [34]:
df_inc.to_csv('../../data-truth/RKI/truth_inc_deaths.csv', index=False)